In [ ]:
import hashlib # TODO - might use a different algorithm? we will see if needed. Start small ffs dont get overwhelemed by the whole ledger

CONFIRMATION_THRESHOLD = 1
REJECTION_THRESHOLD = -1
CONFIRMATION_STEP = 0.1 # TODO - tune this

In [ ]:
class Transaction():
    """
    Transaction containing information to be submitted in the Distributed Ledger.
    Consensus needs to be reached on the validity of the information received.
    """
    def __init__(self, sender_address, recipient_address, sender_private_key, timestamp, tx_data) -> None:
        self.sender_address = sender_address
        self.recipient_address = recipient_address
        self.sender_private_key = sender_private_key
        self.timestamp = timestamp
        self.tx_data = tx_data
        self.confirmation_score: float = 0
        self.is_valid: bool = False
        self.is_rejected: bool = False
        
        # TODO - need to add parent information in here
        
        self.hash = self.calculate_hash()
        
    def calculate_hash(self) -> str:
        """
        Calculates a hash string for a transaction. The hash is generated by encoding all of
        the transaction's unique information, so that ANY change to a transaction results in a new 
        hash, making tampering easier to identify.
        """
        return hashlib.sha256(str(self.sender_address).encode() + self.timestamp.encode() + 
                              self.sender_private_key.encode() + self.tx_data.encode() + 
                              str(self.recipient_address).encode()
                             ).hexdigest()

In [ ]:
class DAG():
    """
    A class representing the Directed Acyclic Graph (DAG) Distributed Ledger Technology.
    When a transaction is received, it is added to the DAG. The number of parents
    for each transaction is decided using a tip selection algorithm.
    """
    
    def __init__(self, consensus_reached: bool) -> None:
        self.chain: list[Transaction] = [self.create_genesis_tx()]
        self.consensus_reached = consensus_reached # Get this value from the consensus class
    
    def create_genesis_tx(self) -> Transaction:
        """
        Creates the genesis transaction to initialise the DAG and provide a parent
        for the first real transaction.
        """
        return Transaction("000", "000", "1234", "24/06/202", "Genesis Transaction")
    
    def add_tx(self, transaction: Transaction) -> None:
        """
        Add a transaction to the DAG
        """
        # TODO Just append for now, sort out tip selection later and number of parents
        # TODO - tx or blocks?? start with tx for now
        self.chain.append(transaction)
        self.check_thresholds(transaction)
        
    def check_thresholds(self, transaction: Transaction) -> None:
        """
        Check if the transaction confirmation or rejection thresholds have been crossed. 
        This will affect weighting. is_valid = strong weighting, else weak weighting
        """
        
        if self.consensus_reached and REJECTION_THRESHOLD <= transaction.confirmation_score <= CONFIRMATION_THRESHOLD:
            transaction.confirmation_score +=  CONFIRMATION_STEP
        else:
            transaction.confirmation_score -= CONFIRMATION_STEP
        
        if transaction.confirmation_score >= CONFIRMATION_THRESHOLD:
            transaction.is_valid = True
        elif transaction.confirmation_score <= REJECTION_THRESHOLD:
            transaction.is_rejected = True

In [ ]:
class ConsensusMechanism():
    """
    The Proof of X consensus mechanism.
    Requires:
    - ID of satellite submission (COSPAR)
    - ID of satellite being witnessed
    - timestamp
    - TLE: epoch, mean motion, eccentricity, inclination, RAAN, perigee, mean anomaly
    
    TODO - need to check: if physically possible, how many times its been seen (maybe new param?)
    # Node needs an ID, trust factor number
    """
    # TODO - proof of Location XYO paper, and bidirectional heuristics (4.3, proof of proximity)
    # TODO - format for data? class for verification/consensus? Need some calcs
    # TODO TODAY - SLAM DRONE PAPER, and consensus on position, doppler shift?? what data so I have to choose from?
    # DAGmap - map consensus could be something to build upon? does it have a ground truth?
    # PowerGraph - consensus for trust level, calculates validity of transacion from probability level. I guess I need to calculate validity from some maths? possible - if yes, hen how accurate/likely
    # probability distruibution for observations? Algorithm one in PowerGraph paper
    pass

# Thoughts
Options are:
1. Satellites send their own positional and trajectory info, along with IDs of any satellites they witness
    - No external measurements needed, just basic witnessing
    - Satellite A reports: ' I saw satellite B and here is my positional data'. Satellite B reports 'I saw satellite A and here is my positional data'. Can verify that these are correct within a certain error tolerance. Is it mathematically possible they saw each other?
2. Satellites send the position and and velocity info of a satellite they witness
    - Requires accurate measurement of another satellite's position and velocity from another satellite. Not sure every satellite can do this. 
    - Stronger witnessing, but larger data set to send. 

- Need to consider doppler shift, TLE data, how can satellites actually witness each other? Through ISLs and broadcasting?
- Could it be verified against existing ground data to begin with? Then moved away with ground as a back up for redundancy?
- What calculations would need to be performed? 
- Incentivised to provide honest information with reputation score, then consensus reached later. 
- Use cases: Invalid info, valid but incorrect info: https://docs.xyo.network/XYO-White-Paper.pdf (Sections 4.2 and 4.3)
- Do I want a constat chain, or a chain that gets queried and then develops? An API? How will users interact? (Maybe a question for later)
- XYO - do I need a signature? What if the node is compromised? I gues smaybe it tells me it is? 


# References 
* https://github.com/samuelxu999/EconLedger/blob/main/src/consensus/ENF_consensus.py
* https://github.com/helium/oracles/blob/main/iot_verifier/src/poc.rs
* https://www.swirlds.com/downloads/SWIRLDS-TR-2016-01.pdf
* https://docs.helium.com/iot/proof-of-coverage-roadmap
* https://www.researchgate.net/publication/379723426_Decentralized_And_Neutral_Consensus_Mechanisms_in_Space_Conjunctions_Assessment_and_Mitigation_Space_DAO_STM
* https://www.sciencedirect.com/science/article/pii/S0094576524003308
* https://www.mdpi.com/2504-446X/6/2/34
* https://www.mdpi.com/1996-1073/12/18/3570
* https://www.linkedin.com/posts/spacecoin-official_proof-of-location-activity-7336127810270375937-TX8U/
* https://spacecoin.org/ 
* decentralized physical infrastructure network: https://ieeexplore.ieee.org/document/10737386
* https://docs.hivemapper.com/, https://geodnet.com/
* https://docs.natix.network/whitepaper/natix-network-ecosystem/data-validation
* location verification first emerged in 2016: https://docs.xyo.network/XYO-White-Paper.pdf (Sections 2, 4.2 and 4.3)